<a href="https://colab.research.google.com/github/vitamingyu/NLP-LLM/blob/main/tf_44attension_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 양방향 LSTM을 사용하는 것이 더 강력합니다. 여기에 추가적으로 어텐션 메커니즘을 사용할 수도 있습니다.

from keras.datasets import imdb
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


17464789/17464789 [==============================] - 0s 0us/step
리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [3]:
# 바다나우 어텐션(Bahdanau Attention)
# 어텐션 스코어 함수란 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수를 말합니다.
# 그리고 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)이었습니다. 다음은 닷 프로덕트 어텐션의 어텐션 스코어 함수를 보여줍니다.

import tensorflow as tf

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


In [4]:
# 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
import os

#  케라스의 함수형 API를 사용합니다. 우선 입력층과 임베딩층을 설계합니다.

sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

# 두번째 층을 설계
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [6]:
# 양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로,
# 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 [==============================] - 458s 4s/step - loss: 0.4736 - accuracy: 0.7636 - val_loss: 0.2836 - val_accuracy: 0.8826
Epoch 2/3
98/98 [==============================] - 463s 5s/step - loss: 0.2397 - accuracy: 0.9145 - val_loss: 0.2856 - val_accuracy: 0.8828
Epoch 3/3
98/98 [==============================] - 419s 4s/step - loss: 0.1900 - accuracy: 0.9361 - val_loss: 0.3303 - val_accuracy: 0.8800


In [8]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 335s 429ms/step - loss: 0.3303 - accuracy: 0.8800

 테스트 정확도: 0.8800


In [9]:
# Attention 후 예측
word_to_index = imdb.get_word_index()
index_to_word = {}

for k, value in word_to_index.items():
  index_to_word [value] = k

import re

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^0-9a-zA-Z ]','', new_sentence).lower()
  encoded = []  # 정수 인코딩
  for word in new_sentence.split():
    try:
      if word_to_index[word] <= 10000:
        encoded.append(word_to_index[word] + 3)   # 0<pad>,1<sos>,2<unk>는 imdb에서 사용
      else:
        encoded.append(2)  # 10000 이상의 값은 <unk> 취급
    except KeyError:
      encoded.append(2)  # 단어 집합에 없는 경우 <unk> 취급

  print('encoded : ', encoded)
  pad_new = pad_sequences([encoded], maxlen=max_len)

  # predict
  score = float(model.predict(pad_new))

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정".format((1 - score) * 100))

print(sentiment_predict("The movie is so much fun, I recommend it."))
print(sentiment_predict("It's boring and dull, and even the screenplay is a mess."))

1641221/1641221 [==============================] - 0s 0us/step
encoded :  [4, 20, 9, 38, 76, 253, 13, 386, 12]
1/1 [==============================] - 6s 6s/step
53.38% 확률로 긍정
None
encoded :  [94, 357, 5, 753, 5, 60, 4, 881, 9, 6, 947]
1/1 [==============================] - 0s 410ms/step
97.17% 확률로 부정
None
